# **Clustering project (phase 3)**
**Contributers:** *Shiva Radmanesh*

\\

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unrar e /content/drive/'My Drive'/DataMining/dataset.rar

In [ ]:
import pandas as pd 
print('done')

\\
#**Part 1 - popularity of the products**

## **Preprocessing**

### make the item-Quantity dataframe

In [ ]:
df_orders = pd.read_csv('orders.csv')
df_orders

In [ ]:
groupedby_item = df_orders.groupby(by=['ID_Item'], as_index=False).sum()
item_quantity = groupedby_item.filter(['ID_Item', 'Quantity_item'], axis=1)

In [ ]:
item_quantity = item_quantity.sort_values(by=['Quantity_item'] ,ascending=False)

item_quantity

\\
### Make the item-likeScore dataframe

In [ ]:
df_keifiat = pd.read_excel('keifiat.xlsx')
df_keifiat

In [ ]:
item_likes_dislikes = df_keifiat.filter(['product_id','likes', 'dislikes'], axis=1)
keifiat_grouped = item_likes_dislikes.groupby(by=['product_id'], as_index=False).sum()

In [ ]:
keifiat_grouped

In [ ]:
keifiat_grouped.columns = ["ID_Item", "likes", "dislikes"]

keifiat_grouped

In [ ]:
keifiat_grouped['like_score'] = keifiat_grouped.apply(lambda row: (row.likes + 1) / (row.dislikes + 1), axis=1)

keifiat_grouped

\\
### combining the previous dataframe to have one dataframe of all needed data
In the code cell below, the two dataframe (item_quantity, keifiat_grouped) are being joined together

In [ ]:
quality_df = item_quantity.join(keifiat_grouped.set_index('ID_Item'), on='ID_Item')

quality_df

\\
**We have 2 approches:**
1. with rows which have NaN values
2. without rows which have NaN values

\\
fill cells which have NaN values with 0

In [ ]:
quality_df_with_NaN = quality_df.fillna(0)

quality_df_with_NaN

In [ ]:
quality_df_with_NaN = quality_df_with_NaN.filter(['Quantity_item', 'likes', 'dislikes', 'like_score', 'like_score'], axis=1)

quality_df_with_NaN

\\
drop cells which have NaN values

In [ ]:
quality_df_without_NaN = quality_df.dropna()

quality_df_without_NaN

In [ ]:
quality_df_without_NaN = quality_df_without_NaN.filter(['Quantity_item', 'likes', 'dislikes', 'like_score'], axis=1)

quality_df_without_NaN

## **Clustering**

### Without NaN values

Agglomerative Clustering

In [ ]:
# agglomerative clustering
from numpy import unique
from numpy import where
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot

# define dataset
X = quality_df_without_NaN.to_numpy()
model = AgglomerativeClustering(n_clusters=4)
# fit model and predict clusters
yhat = model.fit_predict(X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
	# get row indexes for samples with this cluster
	row_ix = where(yhat == cluster)
	# create scatter of these samples
	pyplot.scatter(X[row_ix, 0], X[row_ix, 3])
# show the plot

pyplot.show()

\\
DBSCAN Clustering

In [ ]:
from numpy import unique
from numpy import where
from sklearn.cluster import DBSCAN
from matplotlib import pyplot

X = quality_df_without_NaN.to_numpy()
model = DBSCAN(eps=0.1, min_samples=20)
yhat = model.fit_predict(X)
clusters = unique(yhat)
for cluster in clusters:
	row_ix = where(yhat == cluster)
	pyplot.scatter(X[row_ix, 0], X[row_ix, 3])
pyplot.show()

\\
KMeans Clustering

In [ ]:
# k-means clustering
from numpy import unique
from numpy import where
from sklearn.cluster import KMeans
from matplotlib import pyplot

X = quality_df_without_NaN.to_numpy()
model = KMeans(n_clusters=3)
model.fit(X)
yhat = model.predict(X)
clusters = unique(yhat)
for cluster in clusters:
	row_ix = where(yhat == cluster)
	pyplot.scatter(X[row_ix, 0], X[row_ix, 3])
pyplot.show()

\\
Mini-batch KMeans clustering

In [ ]:
# mini-batch k-means clustering
from numpy import unique
from numpy import where
from sklearn.cluster import MiniBatchKMeans
from matplotlib import pyplot

# define dataset
X = quality_df_without_NaN.to_numpy()
# define the model
model = MiniBatchKMeans(n_clusters=4)
# fit the model
model.fit(X)
# assign a cluster to each example
yhat = model.predict(X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
	# get row indexes for samples with this cluster
	row_ix = where(yhat == cluster)
	# create scatter of these samples
	pyplot.scatter(X[row_ix, 0], X[row_ix, 3])
# show the plot
pyplot.show()

\\
Mean shift clustering

In [ ]:

# mean shift clustering
from numpy import unique
from numpy import where
from sklearn.cluster import MeanShift
from matplotlib import pyplot

# define dataset
X = quality_df_without_NaN.to_numpy()
# define the model
model = MeanShift()
# fit model and predict clusters
yhat = model.fit_predict(X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
	# get row indexes for samples with this cluster
	row_ix = where(yhat == cluster)
	# create scatter of these samples
	pyplot.scatter(X[row_ix, 0], X[row_ix, 3])
# show the plot
pyplot.show()

\\
Gaussian mixture clustering

In [ ]:
# gaussian mixture clustering
from numpy import unique
from numpy import where
from sklearn.mixture import GaussianMixture
from matplotlib import pyplot

# define dataset
X = quality_df_without_NaN.to_numpy()
# define the model
model = GaussianMixture(n_components=4)
# fit the model
model.fit(X)
# assign a cluster to each example
yhat = model.predict(X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
  # get row indexes for samples with this cluster
  row_ix = where(yhat == cluster)
  # create scatter of these samples
  pyplot.scatter(X[row_ix, 0], X[row_ix, 3])
# show the plot
pyplot.show()

\\
### With NaN values

Because most of the rows in our dataset have score, likes and dislikes value equal to NaN and also because we converted NaN values to value equal to 0, there was no enough RAM for calculations to form the desired clusters.

#**Part 2 - clustering the amount of purchases of all pruducts in each city**

## **Preprocessing**

In [ ]:
df_orders = pd.read_csv('orders.csv')
df_orders

\\
Describe the Amount_Gross_Order column

In [ ]:
amount_gross_des = df_orders['Amount_Gross_Order'].describe()
amount_gross_des

In [ ]:
amount_gross_mean = amount_gross_des['mean']
amount_gross_std  = amount_gross_des['std']

\\
Normalizing the Amout_Gross_Order values

In [ ]:
df_orders['normalized_gross'] = df_orders.apply(lambda row: (row.Amount_Gross_Order - amount_gross_mean) / amount_gross_std, axis=1)

df_orders

\\
In the cell below you can see the top 10 cities based on the total number of orders

In [ ]:
groupby_city = df_orders.groupby(by=['city_name_fa']).sum()
sorted = groupby_city.sort_values(by=['Quantity_item'], ascending=False)

In [ ]:
city_quantity = sorted.filter(['Quantity_item', 'normalized_gross'], axis=1)

city_quantity

\\
## **Clustering**

In [ ]:
# agglomerative clustering
from numpy import unique
from numpy import where
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot

# define dataset
X = city_quantity.to_numpy()
# define the model
model = AgglomerativeClustering(n_clusters=3)
# fit model and predict clusters
yhat = model.fit_predict(X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
	# get row indexes for samples with this cluster
	row_ix = where(yhat == cluster)
	# create scatter of these samples
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
# show the plot
pyplot.show()

In [ ]:
from numpy import unique
from numpy import where
from sklearn.cluster import DBSCAN
from matplotlib import pyplot

X = city_quantity.to_numpy()
model = DBSCAN(eps=0.10, min_samples=12)
yhat = model.fit_predict(X)
clusters = unique(yhat)
for cluster in clusters:
	row_ix = where(yhat == cluster)
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
pyplot.show()

In [ ]:
# k-means clustering
from numpy import unique
from numpy import where
from sklearn.cluster import KMeans
from matplotlib import pyplot

X = city_quantity.to_numpy()
model = KMeans(n_clusters=4)
model.fit(X)
yhat = model.predict(X)
clusters = unique(yhat)
for cluster in clusters:
	row_ix = where(yhat == cluster)
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
pyplot.show()

In [ ]:
# mini-batch k-means clustering
from numpy import unique
from numpy import where
from sklearn.cluster import MiniBatchKMeans
from matplotlib import pyplot

# define dataset
X = city_quantity.to_numpy()
# define the model
model = MiniBatchKMeans(n_clusters=4)
# fit the model
model.fit(X)
# assign a cluster to each example
yhat = model.predict(X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
	# get row indexes for samples with this cluster
	row_ix = where(yhat == cluster)
	# create scatter of these samples
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
# show the plot
pyplot.show()

In [ ]:

# mean shift clustering
from numpy import unique
from numpy import where
from sklearn.cluster import MeanShift
from matplotlib import pyplot

# define dataset
X = city_quantity.to_numpy()
# define the model
model = MeanShift()
# fit model and predict clusters
yhat = model.fit_predict(X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
	# get row indexes for samples with this cluster
	row_ix = where(yhat == cluster)
	# create scatter of these samples
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
# show the plot
pyplot.show()

In [ ]:
# gaussian mixture clustering
from numpy import unique
from numpy import where
from sklearn.mixture import GaussianMixture
from matplotlib import pyplot

# define dataset
X = city_quantity.to_numpy()
# define the model
model = GaussianMixture(n_components=4)
# fit the model
model.fit(X)
# assign a cluster to each example
yhat = model.predict(X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
  # get row indexes for samples with this cluster
  row_ix = where(yhat == cluster)
  # create scatter of these samples
  pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
# show the plot
pyplot.show()